<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [1]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap

#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-r5p3h_cq/mightylogic_648ac6103e56402c88bf2c19f87f0f3f
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-r5p3h_cq/mightylogic_648ac6103e56402c88bf2c19f87f0f3f
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=7969 sha256=f3ca181810b56ae4b52c22013f1c179dc1ee54e2cd49240680947b77614ed8f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-16llyvto/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic


# Turf War Demo

In [2]:

#
# A Turf War Map
#

map2 =TurfWarMap()

# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [3]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [4]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [5]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,79.767369,0.000000,51.275865,0.000000,12.373764,61.782278
B,0.000000,121.035331,0.000000,65.405625,99.132865,0.000000
C,79.767369,81.720436,70.403210,130.893111,0.000000,139.174832
D,108.361429,0.000000,131.633495,0.000000,121.853415,0.000000
E,0.000000,59.061723,0.000000,59.622059,12.373764,42.316601


In [6]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

#fig = go.Figure()
#fig.add_traces([fig1.data[0], fig2.data[0]])
#fig.layout.update(fig1.layout)
#fig.layout.update(fig2.layout)

fig1.show()
fig2.show()

In [7]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution



In [8]:
#
# List of buildings easy to parse by players.
#
map2.printBuildingList()

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


# Discrete Solution to Find Next Move

Played around with an analytical solution, but it was too problemmatic.

In [9]:
def scale(target):
  scaler = StandardScaler().fit(target)
  X_scaled = scaler.transform(target)
  X_scaled = 1 * (X_scaled + 1)
  return(X_scaled)

In [10]:
#
# High values are desirable payout locations, low are less desirable
#
scaled_payout = scale(payout)
super_threshold_indices = scaled_payout < 0.1
scaled_payout[super_threshold_indices] = 0

pd.DataFrame(scaled_payout)

,0,1,2,3,4,5
0,0.5,0.000000,0.000000,0.000000,0.000000,0.256737
1,3.0,1.320660,2.499776,1.185935,1.126549,2.769380
2,0.5,1.320660,1.364708,1.185935,2.731603,0.256737
3,0.5,2.548282,1.364708,2.669610,1.126549,1.460409
4,0.5,0.000000,0.000000,0.151649,0.000000,0.256737


In [11]:
#
# High values are desirable staging locations, low are less desirable
#
scaled_staging = scale( staging)/3

super_threshold_indices = scaled_staging < 0.1
scaled_staging[super_threshold_indices] = 0

pd.DataFrame(scaled_staging)

,0,1,2,3,4,5
0,0.527423,0.000000,0.337493,0.000000,0.000000,0.418689
1,0.000000,0.819020,0.000000,0.430641,0.661306,0.000000
2,0.527423,0.540962,0.467208,0.878724,0.000000,0.921899
3,0.739344,0.000000,0.882449,0.000000,0.810382,0.000000
4,0.000000,0.380707,0.000000,0.391068,0.000000,0.292122


In [12]:
#
# Highest values are desirable staging locations, lowest are desirable payout locations
#
adjusted_move = scaled_payout+scaled_staging
pd.DataFrame(adjusted_move)

,0,1,2,3,4,5
0,1.027423,0.000000,0.337493,0.000000,0.000000,0.675426
1,3.000000,2.139680,2.499776,1.616576,1.787856,2.769380
2,1.027423,1.861622,1.831915,2.064659,2.731603,1.178636
3,1.239344,2.548282,2.247157,2.669610,1.936931,1.460409
4,0.500000,0.380707,0.000000,0.542717,0.000000,0.548859


In [13]:
moves = adjusted_move

np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 250
get_heatmap(moves,"Movement Array", round=False).show()
moves

array([[1.02742312, 0.        , 0.33749286, 0.        , 0.        ,
        0.67542627],
       [3.        , 2.13967973, 2.49977623, 1.61657576, 1.78785556,
        2.76937953],
       [1.02742312, 1.86162233, 1.83191536, 2.06465919, 2.7316031 ,
        1.17863643],
       [1.23934409, 2.54828178, 2.24715713, 2.66961021, 1.93693141,
        1.46040928],
       [0.5       , 0.380707  , 0.        , 0.54271688, 0.        ,
        0.5488595 ]])

In [14]:
flip = moves
#flip = np.flip( moves, axis=0) 
flip = np.flip( flip, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])

fig.update_layout(title='Movement Array in 3D', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [20]:

flip = moves
epsilon = 0.05
rows = range(5)
cols = range(6)
retx = [[0 for i in cols] for j in rows]
rety = [[0 for i in cols] for j in rows]

for row in rows:
  for col in cols:
    up = row +1
    down = row -1
    left = col -1
    right = col +1

    me = flip[row][col]
    max = me
    z_up = -1000000
    z_dn = -1000000
    z_left = -1000000
    z_right = -1000000

    if up in rows:
      tmp = flip[up][col]
      z_up = tmp
      if tmp> max:
        max = tmp
    if down in rows:
      tmp = flip[down][col]
      z_dn = tmp
      if tmp> max:
        max = tmp        
    if left in cols:
      tmp = flip[row][left]
      z_left = tmp
      if tmp> max:
        max = tmp     
    if right in cols:
      tmp = flip[row][right]
      z_right= tmp
      if tmp> max:
        max = tmp     

    drow = 4-row
    if max == me:
      retx[drow][col] = 0
      rety[drow][col] = 0
    elif max == z_left:
      retx[drow][col] = -1
      rety[drow][col] = 0      
    elif max == z_right:
      retx[drow][col] = 1
      rety[drow][col] = 0    
    elif max == z_up:
      retx[drow][col] = 0
      rety[drow][col] = -1    
    elif max == z_dn:
      retx[drow][col] = 0
      rety[drow][col] = 1  



In [21]:
pd.DataFrame(retx)

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,1,0,1,0,0,-1
2,0,0,0,1,0,0
3,0,-1,0,-1,1,0
4,0,0,0,0,0,0


In [22]:
pd.DataFrame(rety)

,0,1,2,3,4,5
0,1,1,1,1,1,1
1,0,0,0,0,1,0
2,1,-1,1,0,0,1
3,0,0,0,0,0,0
4,-1,-1,-1,-1,-1,-1


In [39]:
  #fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  x,y = np.meshgrid(np.arange(0, 6, 1), np.arange(0, 5, 1))
  fig = ff.create_quiver(x,y, retx, rety, scale=1, name="Recommended Move") #, x_start=0, x_end=6)


  cnames = []
  rtl = ['A','B','C','D','E','F']
  ctli = [5,4,3,2,1]
  for r in rtl:
    for c in ctli:
      cnames.append(f"{r}{c}")
      #print(f"{r}{c}")

  # add dots at integers
  fig.add_trace(go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers+text',
                          textposition="top right",
                          text = cnames,
                          marker_size=14,
                          name='Tiles'))
  fig.show()
  cnames

['A5',
 'A4',
 'A3',
 'A2',
 'A1',
 'B5',
 'B4',
 'B3',
 'B2',
 'B1',
 'C5',
 'C4',
 'C3',
 'C2',
 'C1',
 'D5',
 'D4',
 'D3',
 'D2',
 'D1',
 'E5',
 'E4',
 'E3',
 'E2',
 'E1',
 'F5',
 'F4',
 'F3',
 'F2',
 'F1']

# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

